# 地铁网络数据


In [1]:
import pandas as pd
import networkx as nx

In [2]:
import sys
sys.path.append('../k_libs')
from subway_network import NetDataPivot, NetDegree, NetDataGraph, NetLength, CityNetworkAnalyzer

# 数据处理

In [3]:
df_row = pd.read_csv('../data/subway/ads_subway_cleaned.csv')
df_row

,city_id,city_name,city_pinyin,line_number_x,line_name,line_full_name,line_color,line_id,line_is_loop,line_is_show,...,target_st_name,target_st_longitude,target_st_latitude,st_cnt,uni_line_number,longitude_bd,latitude_bd,target_st_longitude_bd,target_st_latitude_bd,ll_distance
0,1100,北京,beijing,8,7号线,地铁7号线,F9BE58,110100023054,0,1,...,花庄,116.695495,39.856722,2,8,116.685663,39.854994,116.702126,39.862387,1.633
1,1100,北京,beijing,8,7号线,地铁7号线,F9BE58,110100023054,0,1,...,高楼金,116.684340,39.863154,2,8,116.702126,39.862387,116.690939,39.868921,1.191
2,1100,北京,beijing,8,7号线,地铁7号线,F9BE58,110100023054,0,1,...,群芳,116.670792,39.863515,1,8,116.690939,39.868921,116.677315,39.869507,1.157
3,1100,北京,beijing,8,7号线,地铁7号线,F9BE58,110100023054,0,1,...,万盛东,116.657379,39.863491,1,8,116.677315,39.869507,116.663823,39.869712,1.145
4,1100,北京,beijing,8,7号线,地铁7号线,F9BE58,110100023054,0,1,...,万盛西,116.632160,39.863328,1,8,116.663823,39.869712,116.638568,39.869655,2.153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7477,8200,澳门,aomen,3,氹仔线,氹仔线,008187,900000126430,0,1,...,马会,113.552300,22.154190,1,3,113.563410,22.157302,113.558795,22.160146,0.565
7478,8200,澳门,aomen,2,石排湾线,石排湾线,786DAF,900000213458,0,1,...,NaN,NaN,NaN,1,2,113.573270,22.134229,NaN,NaN,NaN
7479,8200,澳门,aomen,2,石排湾线,石排湾线,786DAF,900000213458,0,1,...,石排湾,113.566708,22.128488,2,2,113.574886,22.141501,113.573270,22.134229,0.828
7480,8200,澳门,aomen,1,横琴线,横琴线,E3002A,900000213462,0,1,...,NaN,NaN,NaN,1,1,113.556959,22.142970,NaN,NaN,NaN


# 可视化数据


## 网络特征


In [8]:
df_index = pd.read_csv('../data/subway/ads_subway_index.csv')
df_index


,序号,城市,线路数,车站数,连边数,平均度,最大度,换乘站比例,密度,同配系数,平均聚类系数,是否连通图,直径,平均最短路径长度,全局效率,平均局部效率
0,1,北京,27,401,480,2.39,6,0.2344,0.00599,0.10849,0.00648,是,50,15.36,0.09472,0.00648
1,2,上海,21,413,496,2.40,8,0.2131,0.00583,0.19857,0.01356,是,44,15.05,0.09480,0.01384
2,3,广州,21,322,375,2.33,5,0.2143,0.00726,0.06376,0.00466,是,48,15.67,0.09588,0.00466
3,4,深圳,17,335,387,2.31,8,0.1761,0.00692,0.11148,0.00333,否,40,14.95,0.09933,0.00380
4,5,成都,15,323,370,2.30,6,0.1801,0.00716,-0.02500,0.00155,是,41,14.47,0.09998,0.00155
5,6,重庆,15,268,299,2.24,6,0.1610,0.00842,0.07740,0.00437,是,49,15.80,0.09782,0.00437
6,7,杭州,14,296,329,2.22,6,0.1554,0.00754,-0.04247,0.00225,是,64,18.39,0.08844,0.00225
7,8,南京,13,226,242,2.16,6,0.1250,0.00969,-0.03980,0.00000,是,49,16.25,0.09700,0.00000
8,9,郑州,13,235,271,2.31,6,0.2000,0.00986,-0.10257,0.00213,否,34,10.94,0.13319,0.00262
9,10,武汉,12,271,298,2.20,6,0.1365,0.00815,-0.00522,0.00332,是,46,15.99,0.09490,0.00332


## 网络图
经纬度坐标图，信息可视化坐标图


In [5]:
city_name = "成都"

In [7]:
df_city = df_row[df_row['city_name'] == city_name]
df_city

,city_id,city_name,city_pinyin,line_number_x,line_name,line_full_name,line_color,line_id,line_is_loop,line_is_show,...,target_st_name,target_st_longitude,target_st_latitude,st_cnt,uni_line_number,longitude_bd,latitude_bd,target_st_longitude_bd,target_st_latitude_bd,ll_distance
6411,5101,成都,chengdu,1,1号线,地铁1号线,2249A3,510100022320,0,1,...,火车南站,104.067393,30.605459,2,1,104.070312,30.600465,104.073886,30.611503,1.279
6412,5101,成都,chengdu,1,1号线,地铁1号线,2249A3,510100022320,0,1,...,倪家桥,104.066885,30.624773,2,1,104.073615,30.621361,104.073389,30.630829,1.052
6413,5101,成都,chengdu,1,1号线,地铁1号线,2249A3,510100022320,0,1,...,天府广场,104.065751,30.657453,2,1,104.072544,30.656113,104.072233,30.663523,0.824
6414,5101,成都,chengdu,1,1号线,地铁1号线,2249A3,510100022320,0,1,...,高新,104.063847,30.594368,2,1,104.070604,30.588669,104.070312,30.600465,1.311
6415,5101,成都,chengdu,1,1号线,地铁1号线,2249A3,510100022320,0,1,...,升仙湖,104.083474,30.704625,3,1,104.080557,30.701446,104.090020,30.710418,1.355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6790,5101,成都,chengdu,4,S3(资阳)线,S3(资阳)线,B5B5B6,900000156039,0,1,...,苌弘广场,104.650507,30.109039,1,4,104.628290,30.132626,104.656920,30.115304,3.369
6791,5101,成都,chengdu,4,S3(资阳)线,S3(资阳)线,B5B5B6,900000156039,0,1,...,资阳临空,104.584953,30.172822,3,4,104.393951,30.284414,104.591548,30.178521,22.295
6792,5101,成都,chengdu,4,S3(资阳)线,S3(资阳)线,B5B5B6,900000156039,0,1,...,幸福大道,104.621836,30.126442,1,4,104.591548,30.178521,104.628290,30.132626,6.259
6793,5101,成都,chengdu,4,S3(资阳)线,S3(资阳)线,B5B5B6,900000156039,0,1,...,资阳北站,104.681450,30.126670,1,4,104.680596,30.120141,104.688008,30.132495,1.556


## 直径图
部分城市的地铁网络并非严格意义上的连通图。例如，长沙的磁悬浮机场线（S2 线）以及深圳的坪山云巴 1 号线均为独立线路，与城市主体地铁网并未形成连通。因此，在进行地铁网络建模与计算结构性指标时，通常需要基于最大连通子图进行分析，以避免孤立子图对平均最短路径、网络效率等指标造成干扰或失真。
本研究中将各类轨道交通线路统一统称为“地铁”。在若干城市的地铁系统中，整体网络并非严格意义上的单一连通图。例如，长沙的 S2 线（机场线）以及深圳的坪山云巴 1 号线均为独立运行线路，与城市主体地铁网络缺乏连接。若直接将这些孤立子网纳入整体网络建模，可能导致部分拓扑指标（如平均最短路径长度、全局效率等）产生偏差，从而影响对网络整体特性的准确刻画。因此，本研究在计算地铁网络的结构性指标时，统一基于最大连通子图进行分析，以确保所得结果能够更真实地反映城市地铁主体网络的结构特征与运行效率。
